# Activation Counting Tools

This notebook is to aid in determining optimal count times given a set of activations and experiments design parameters.
It reads from a formatted excel input to obtain the data for multiple foils and plots the count time versus the irradiation time.

Currently has constant background and efficiency terms. This needs to be updated.  

Import the following packages:

In [1]:
import openpyxl

import numpy as np
import math as m

from pyne import data
from pyne import nucname
from scipy.integrate import quad
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: QAWarning: pyne.data is not yet QA compliant.


### Required Functions

Functions and methods required for notebook calculations

In [2]:
def CalcN_0(par, n, t, rate, src, vol, tt=0.0):
    """
    Calculates the initial population of atoms post-irradiation  
   
    Parameters
    ==========
    par : PyNE nucname string
        Id of the parent isotope
    n: int
        Initial number of parent atoms from previous irradiations
    t: int
        Irradiation time in sec
    rate : float
        The reaction rate in rx per sec per src per cm3
    src : float
        The source strength in n per sec    
    vol : float
        The volume of the foil in cm3  
    tt: int
        Post-irradiation transfer time in sec

    Optional
    ========
        
    Returns
    =======
    N_0 float
        The number of atoms after irradiation time t 
    """ 
    
    if par=="In116M":
        N_0=rate*vol*src/2.12792E-4*(1-m.exp(-2.12792E-4*t))+n*m.exp(-2.12792E-4*t)
        N_0=N_0*m.exp(-2.12792E-4**tt)
    else:
        N_0=rate*vol*src/data.decay_const(par)*(1-m.exp(-data.decay_const(par)*t))+n*m.exp(-data.decay_const(par)*t)
        N_0=N_0*m.exp(-data.decay_const(par)*tt)
    
    return N_0

#-------------------------------------------------------------------------------------------------------------#
def Activity(t, par, n):
    """
    Calculates the activity of a given isotope at time t after the irradiation.  
   
    Parameters
    ==========
    t: int
        Time post irradiation
    par : PyNE nucname string
        Id of the parent isotope
    n: int
        Initial number of parent atoms

    Optional
    ========
        
    Returns
    =======
    act : float
        The activity at time t in decays/s
    """ 
    
    if par=="In116M":
        act=2.12792E-4*n*m.exp(-2.12792E-4*t)
    else:
        act=data.decay_const(par)*n*m.exp(-data.decay_const(par)*t)
    
    return act

#-------------------------------------------------------------------------------------------------------------#
def GCF(r_src, r_det, det2src):
    """
    Calculates the solid angle geometry correction factor for the detector configuration from Knoll p. 119  
   
    Parameters
    ==========
    r_det : float
        Radius of the detector in cm
    r_src: float
        Radius of the foil in cm
    det2src: float
        Distance from the detector to src in cm 

    Optional
    ========
        
    Returns
    =======
    gcf : float
        The gcf for the given configuration
    """ 

    alpha=(r_src/det2src)**2
    beta=(r_det/det2src)**2
    f1=5./16.*(beta/(1+beta)**(7./2.))-35./64.*(beta**2/(1+beta)**(9./2.))
    f2=35./128.*(beta/(1+beta)**(9./2.))-315./256.*(beta**2/(1+beta)**(11./2.))+1155./1028.*(beta**3/(1+beta)**(13./2.))
    gcf=0.5*(1-1./(1+beta)**(1./2.)-3./8.*(alpha*beta/(1+beta)**(5./2.))+alpha**2*f1-alpha**3*f2)

    return gcf

#-------------------------------------------------------------------------------------------------------------#
def CalcRelEff(e,det2src):
    """
    Calculates the relative efficiency based on calibration curves at a given distance  
   
    Parameters
    ==========
    e : float
        Incident gamma ray energy in keV
    det2src: float
        Distance from the detector to src in cm 

    Optional
    ========
        
    Returns
    =======
    eff : float
        The relative efficienty for the given configuration and line
    """ 
    
    if det2src==0:
        eff=10**(0.6368*10-0.1175*10*m.log10(energy)+0.9785*0.1*m.log10(energy)**2-(0.3278*10**4)/(energy**2))
    elif det2src==1:
        eff=10**(0.5166*10-0.4065*m.log10(energy)-0.4308*0.1*m.log10(energy)**2-0.1888*10**4/energy**2)
    elif det2src==2:
        eff=10**(0.5722*10-0.8004*m.log10(energy)+0.1418*0.1*m.log10(energy)**2-0.2112*10**4/energy**2)
    elif det2src==3:
        eff=10**(0.585*10-0.9388*m.log10(energy)+0.3163*0.1*m.log10(energy)**2-0.2266*10**4/energy**2)
    elif det2src==4:
        eff=10**(0.5736*10-0.9279*m.log10(energy)+0.2328*0.1*m.log10(energy)**2-0.2373*10**4/energy**2)
    elif det2src==5:
        eff=10**(0.6114*10-0.1225*10*m.log10(energy)+0.6998*0.1*m.log10(energy)**2-0.2768*10**4/energy**2)
    else:
        print "No calibration exists for that detector to source distance"

    return eff

## User Experimental Input

Specify the experimental parameters determining the reaction rate.  This assumes that the T<sub>1/2</sub> is >> 1 seconds, which should be true for all practical experiments.

### Counting Facility Variables
background = the background rate at the peak of interest in counts/s

det_r = radius of the detector in cm

det_foil_dist = the distance from the detector face to the foil in cm

eff = the detection efficiency for a given gamma

sigma = the desired counting statistics level

trans_t= the time to transfer the foil to the counting facility following irradiation in sec

### File Name
fname = path of the file containing the foil and beam data

In [3]:
background=0.01
det_r=5.08
det2foil_dist=1
#eff=0.5
sigma=0.01
trans_t=300

fname='/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Simulated/88Inch_Activation/88_Vault_ETA_IRDFF/Foils.xlsx'  

### Beam Variables
src = the neutron source strength in in n/sec

### Foil Variables
rx_prod = name of the foil reaction product written in the format "XXAAA" - ex "U235" or "Rb86"

rx_rate = the reaction rate per source particle in units of reactions/cm<sup>3</sup>/src (this can be obtained from simulation or a simple calculation). It assumes that the natural abundance of the isotope is accounted for in this rate.  

foil_r = the foil radius in cm

foil_h = the foil height in cm

vol = the foil volume in cm<sup>3</sup>

In [4]:
# Define the activity integrand accounting for all of the efficiencies
def integrand(t):
     return Activity(t,rx_prod,N_0)*abs_eff/100*BR/100

# Load Excel file and read all rows and tabs
wb = openpyxl.load_workbook(fname, data_only=True)
for tab in wb.get_sheet_names():
    # Import sheet object
    sheet = wb.get_sheet_by_name(tab)

    # Read in data
    i=2   #Skip the header
    plot_data=[]
    plot_labels=[]
    while sheet.cell(row=i, column=1).value != None:
        rx=str(sheet.cell(row=i, column=1).value)
        rx_prod=str(sheet.cell(row=i, column=2).value)
        energy=float(sheet.cell(row=i, column=3).value)
        BR=float(sheet.cell(row=i, column=4).value)
        src=float(sheet.cell(row=i, column=5).value)
        rx_rate=float(sheet.cell(row=i, column=6).value)
        foil_r=float(sheet.cell(row=i, column=8).value)
        foil_h=float(sheet.cell(row=i, column=9).value)
        
        # Calculate the foil volume
        vol=np.pi*foil_r**2*foil_h
        i+=1
        
        # Calculate the GCF
        gcf=GCF(foil_r,det_r,det2foil_dist)
        
        # Calculate absolute efficiency
        abs_cf=[(0,95775.80,4.7077),(1,6016.50,2.6323),(2,5694.63,1.952),(3,4392.66,1.4806),(4,3172.02,1.0245),(5,2643.60,0.8170)]
        ref_rel_eff=abs_cf[det2foil_dist][1]
        rel_eff=CalcRelEff(energy,det2foil_dist)
        abs_eff=rel_eff/ref_rel_eff*abs_cf[det2foil_dist][2]
        
        # Calculate the count time for a given irradiation time and store the data
        timeprs=[]
        for src_t in range(180,3*86400,120):    # Irradiation time in seconds
            # Calculate the initial number of atoms from the irradiation (assumes natural abundance is captured in rx_rate)
            N_0=CalcN_0(rx_prod, 0.0, src_t, rx_rate, src, vol, trans_t)
            
            # Approximate the optimal foil counting time using an average count rate
            tf=1
            diff=1000
            try:
                while diff > 1:
                    prevt=tf
                    S = quad(integrand, 0, tf)[0]/tf
                    tf=((m.sqrt(S+background)+m.sqrt(background))**2/(sigma**2*S**2))/(1+1/m.sqrt((S+background)/background))  #Knoll eqn 3.54/55
                    diff=tf-prevt
            except ZeroDivisionError:
                tf=1E99
            timeprs.append([src_t,tf]) 
        
        # Append to the data arrays
        plot_data.append(timeprs)
        plot_labels.append(rx)

/usr/local/lib/python2.7/dist-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


### Plots

Bringing it all together....

In [5]:
# Allow use of Tex sybols
plt.rc('text', usetex=True)

# Set up figure
fig = plt.figure()
#ax1 = fig.add_subplot(111)
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.85])

# Preset data set format scheme
s=10
linewidth=['1.5','2.5','3.5','4.5']
marker=['.','o','v','^','<','>','1','2','3','4','8','s','p','*','h','H','+','x','d','D']
linestyle=[':', '-.', '--', '-','-']
dashes=[[2, 2, 2, 2], [10, 5, 2, 5], [10, 5, 10, 5], [10, 2, 2, 2, 2, 2],
        [10, 0.1]]

# Set Line color cycle
ax1.set_color_cycle(['k', 'k', 'k', 'k', 'k', 'r', 'r', 'r','r', 'r', 'b', 'b', 'b', 'b', 'b', 'grey', 'grey', 'grey', 'grey', 'grey'])
#ax1.set_color_cycle(['k','b','g','r','c','m','y','w'])

# Set axes
ax1.axis([1, 3*1450, 1, 5*2900])
ax1.set_xscale('log')
ax1.set_yscale('log')

# Set axes labels and plot title.
# ax1.set_title("Plot title...")    
ax1.set_xlabel('\\textbf{Irradiation Time [min]}', fontsize=18, weight="bold")
ax1.set_ylabel('\\textbf{Count Time [min]}', fontsize=18, weight="bold")
ax1.tick_params(axis='both', which='major', labelsize=18, width=2)
ax1.tick_params(axis='both', which='minor', width=2)

# Loop over the list of data
for i in range(len(plot_labels)):
    # Reset x and y data lists and index for lists
    x=[]
    y=[]

    # Build list of x, y coord pairs  
    for d in plot_data[i]:
        x.append(d[0]/60) 
        y.append(d[1]/60)

    # Add data set to line plot
    ax1.plot(x, y, linewidth=linewidth[i//5], linestyle=linestyle[i%5], 
                label=plot_labels[i], dashes=dashes[i%5]) 

    # Add and locate legend
    leg = ax1.legend()
    plt.legend(borderaxespad=0.75, loc=2, fontsize=16, handlelength=5, borderpad=0.5,\
                labelspacing=0.75, fancybox=True, framealpha=0.5);

plt.show()